Написать приложение и функцию, которые собирают основные новости с сайта на выбор dzen.ru, lenta.ru, mail.ru . Для парсинга использовать XPath
Структура данных должна содержать:
* название источника
* наименование новости
* ссылку на новость
* дата публикации

минимум один сайт максимум все

Задание считается выполненным, если:
- код задокументирован, парсер работает корректно
- репозиторий в github имеет readme.md с описанием и структурой проекта (если сдача github, gitlab )
- есть скриншоты результата выполнения задания

ДЗ принимается ссылками на github, gitlab, colab

## Подключение необходимых библиотек

In [44]:
import pandas as pd
import requests
from lxml import html
import os

## Объявление и инициализация констант

In [22]:
REQUIRED_SITE_NEWS_DZEN_URL = "https://lenta.ru"
REQUIRE_HEADER = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/110.0"
}

In [23]:
def get_page_content(url, header):
    '''
    Формирование контента страницы в виде строки
    '''
    page_data = requests.get(url, header)
    
    if not page_data.ok:
        raise Exception(page_data.status_code)
    
    return page_data.text

In [37]:
def get_all_records(dom_obj):
    'Формирование списка всех важных новостей'
    return dom_obj.xpath("//div[@class='last24']/a")

In [41]:
def get_links(all_records):
    links = []
    for record in all_records:
        links.append(record.xpath('@href')[0])
    return links

## Представление страницы в виде строки

In [30]:
page_data = get_page_content(REQUIRED_SITE_NEWS_DZEN_URL, REQUIRE_HEADER)

## Преобразование полученной строки к DOM-объекту

In [31]:
page_dom = html.fromstring(page_data)

## Поиск требуемых данных

In [38]:
all_records = get_all_records(page_dom)

In [42]:
links = get_links(all_records)

In [47]:
links = list(map(lambda link: os.path.join(REQUIRED_SITE_NEWS_DZEN_URL, link), links))

In [48]:
links

['/news/2023/02/17/voennye_okruga/',
 '/news/2023/02/17/nemovv/',
 '/news/2023/02/17/minobr/',
 '/news/2023/02/17/kovtunn/',
 '/news/2023/02/17/bplabryansk/',
 '/news/2023/02/17/fsbbb/',
 '/news/2023/02/17/bpla/',
 '/news/2023/02/17/posolstvo_nuland/']